In [7]:
import autokeras as ak

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

%run utils.ipynb

In [8]:
def _get_padding(kernel_size, output_node):
    if all([kernel_size * 2 <= length
            for length in output_node.shape[1:-1]]):
        return 'valid'
    return 'same'

In [ ]:
def build_cnn_model(
    input_shape,
    output_shape,
    normalize=True,
    kernel_size=3,
    num_blocks=2,
    num_layers=2,
    filters=(32, 32, 32, 32),
    separable=False,
    max_pooling=True,
    dropout_rate=0,
    batchnorm=False,
):
    """Build an vanilla CNN regression model with option optional separable blocks.
    Code based on Autokeras.
    """

    assert len(input_shape) == 3, "The input images should have a channel dimension"
    assert len(filters) == num_blocks * num_layers
    inputs = layers.Input(shape=input_shape)
    x = inputs

    if normalize:
        # Compute the mean and the variance of the dataset and store it as model weights.
        # Don't forget to use adapt_model(model, X) before fitting the model.
        x = preprocessing.Normalization()(x)

    if separable:
        conv = ak.utils.get_sep_conv(inputs.shape)
    else:
        conv = ak.utils.get_conv(inputs.shape)

    pool = ak.utils.get_max_pooling(inputs.shape)

    counter = 0
    for i in range(num_blocks):
        for j in range(num_layers):
            x = conv(filters[counter], kernel_size,
                               padding=_get_padding(kernel_size, x),
                               activation='relu')(x)
            counter += 1
        if max_pooling:
            x = pool(kernel_size - 1,
                               padding=_get_padding(kernel_size - 1, x))(x)
        if dropout_rate > 0:
            x = layers.Dropout(dropout_rate)(x)
        
        if batchnorm:
            x = layers.BatchNormalization()(x)
    
    # Regression head
    x = layers.Flatten()(x)
    outputs = layers.Dense(output_shape[-1])(x)
    model = Model(inputs, outputs)
    
    return model

In [1]:
# Usage example:
# model = build_cnn_model(input_shape=(64, 64, 1), output_shape=(5,), num_blocks=3, separable=True,
#                        filters=(16, 16, 16, 512, 512, 16), batchnorm=True)
# adapt_model(model, X)
# model.summary()